In [24]:
import pandas as pd
import geopandas as gpd
import tqdm
from google.cloud import bigquery
from shapely import wkt
from pathlib import Path

In [25]:
PROJECT_ID = "tm-geodata-warehouse"
DATA_DIR = Path("../../data/")

## GCP Authentication
If necessary, authenticate GCP using this command `gcloud auth login --update-adc`

In [3]:
bq_client = bigquery.Client(project=PROJECT_ID)

## Extract target areas' admin boundaries

In [29]:
%%time
query = """
SELECT ADM1_EN,
       ADM1_PCODE,
       ADM2_EN,
       ADM2_PCODE,
       ADM3_EN,
       ADM3_PCODE,
       ADM4_EN,
       ADM4_PCODE,
       ST_GEOGFROMTEXT(geometry) AS geometry
FROM  tm-geodata-warehouse.ph_admin_boundary_dec2017.level4_barangay 
WHERE ADM3_EN = 'City of Navotas' OR 
      ADM3_EN = 'City of Mandaluyong' OR
      ADM3_EN = 'City of Muntinlupa' OR 
      ADM3_EN = 'Dagupan City' OR
      ADM3_EN = 'Palayan City' OR
      ADM3_EN = 'Legazpi City' OR
      ADM3_EN = 'Iloilo City' OR
      ADM3_EN = 'Mandaue City' OR
      ADM3_EN = 'Tacloban City' OR
      ADM3_EN = 'Zamboanga City' OR
      ADM3_EN = 'Cagayan de Oro City' OR
      ADM3_EN = 'Davao City'
"""
admin_bound_targets = pd.read_gbq(query, PROJECT_ID, progress_bar_type="tqdm")
admin_bound_targets.head()

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2007: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


CPU times: user 195 ms, sys: 20.5 ms, total: 215 ms
Wall time: 17.7 s


,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON((120.327417356 16.0542344840001, 120.3..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON((120.333803161 16.039738444, 120.33369..."
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON((120.347368144 16.060093308, 120.34721..."
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay I (T. Bugallon),PH015518024,"POLYGON((120.340538921 16.0448877850001, 120.3..."
4,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,Imelda Valley,PH034919017,"POLYGON((121.122502152 15.5802818290001, 121.1..."


In [33]:
admin_bound_targets["ADM3_EN"].unique()

array(['Dagupan City', 'Palayan City', 'Legazpi City', 'Iloilo City',
       'Mandaue City', 'Tacloban City', 'Zamboanga City',
       'Cagayan de Oro City', 'Davao City', 'City of Mandaluyong',
       'City of Navotas', 'City of Muntinlupa'], dtype=object)

In [31]:
admin_bound_targets.groupby("ADM3_EN")["ADM4_EN"].count()

ADM3_EN
Cagayan de Oro City     80
City of Mandaluyong     27
City of Muntinlupa       9
City of Navotas         14
Dagupan City            31
Davao City             183
Iloilo City            180
Legazpi City            70
Mandaue City            27
Palayan City            19
Tacloban City          138
Zamboanga City         101
Name: ADM4_EN, dtype: int64

In [34]:
admin_bound_targets["geometry"] = admin_bound_targets["geometry"].apply(wkt.loads)
gdf = gpd.GeoDataFrame(admin_bound_targets, crs="epsg:4326")

In [35]:
gdf.to_file(DATA_DIR / "target_admin_bounds.shp")

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
